In [38]:
import pickle
import pandas as pd
import numpy as np

In [8]:
# open a file, where you stored the pickled data
file = open('wine_word2vec_model', 'rb')

# dump information to that file
wine_word2vec_model = pickle.load(file)

# close the file
file.close()

# Same for the rest of the files
file = open('model_knn', 'rb')
model_knn = pickle.load(file)
file.close()

file = open('dict_of_tfidf_weightings', 'rb')
dict_of_tfidf_weightings = pickle.load(file)
file.close()

file = open('wine_reviews_mincount', 'rb')
wine_reviews_mincount = pickle.load(file)
file.close()

In [9]:
def descriptors_to_best_match_wines(list_of_descriptors, number_of_suggestions=10):
    weighted_review_terms = []
    for term in list_of_descriptors:
        if term not in dict_of_tfidf_weightings:
            if term not in descriptor_mapping.index:
                print('choose a different descriptor from', term)
                continue
            else:
                term = descriptor_mapping['normalized'][term]
        tfidf_weighting = dict_of_tfidf_weightings[term]
        word_vector = wine_word2vec_model.wv.get_vector(term).reshape(1, 300)
        weighted_word_vector = tfidf_weighting * word_vector
        weighted_review_terms.append(weighted_word_vector)
    review_vector = sum(weighted_review_terms)

    distance, indice = model_knn.kneighbors(review_vector, n_neighbors=number_of_suggestions+1)
    distance_list = distance[0].tolist()[1:]
    indice_list = indice[0].tolist()[1:]

    n = 1
    for d, i in zip(distance_list, indice_list):
        wine_name = wine_reviews_mincount['title'][i]
        wine_descriptors = wine_reviews_mincount['normalized_descriptors'][i]
        print('Suggestion', str(n), ':', wine_name, 'with a cosine distance of', "{:.3f}".format(d))
        print('This wine has the following descriptors:', wine_descriptors)
        print('')
        n+=1

In [10]:
descriptors = ['apple','pear','wood']
descriptors_to_best_match_wines(list_of_descriptors=descriptors, number_of_suggestions=5)

Suggestion 1 : Domaine Faiveley 2011 Les Villeranges  (Rully) with a cosine distance of 0.084
This wine has the following descriptors: ['fresh', 'fruit', 'citrus', 'pear', 'apple', 'wood', 'round']

Suggestion 2 : Domaine Ostertag 2010 Barriques Pinot Blanc (Alsace) with a cosine distance of 0.088
This wine has the following descriptors: ['oak', 'wood', 'toast', 'pear', 'honey', 'apple']

Suggestion 3 : A.R. Lenoble  NV Intense Brut  (Champagne) with a cosine distance of 0.110
This wine has the following descriptors: ['wood', 'rich', 'ripe', 'fruit', 'pear', 'crisp', 'apple', 'apricot']

Suggestion 4 : Domaine de la Garenne 2008  Mâcon-Azé with a cosine distance of 0.120
This wine has the following descriptors: ['wood', 'fruit', 'fresh', 'grapefruit', 'pear', 'bright', 'lemon', 'honey', 'wood']

Suggestion 5 : Wheelhouse 2011 Chardonnay (Sonoma County) with a cosine distance of 0.126
This wine has the following descriptors: ['ripe', 'apple', 'pear', 'tropical_fruit', 'fruit', 'oak']



In [11]:
wine_reviews_mincount

,level_0,index,country,description,designation,points,price,province,region_1,title,variety,winery,normalized_descriptors,review_vector,descriptor_count
0,0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,"[tropical_fruit, fruit, dry, herb, apple, citr...","[[-0.37825504, -5.1521606, -1.7322228, -2.4307...",9
1,1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,"[ripe, fruit, smooth, firm, juicy, berry, frui...","[[-1.2453364, -3.138025, 1.7377969, -0.0509741...",8
2,2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,"[tart, snappy, lime, green, pineapple, crisp, ...","[[-5.1481357, -4.3782115, 0.27906758, 0.112504...",7
3,3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,"[pineapple, rind, lemon_pith, orange_blossom, ...","[[-1.2961392, -2.4018247, 1.1718184, 0.2752304...",6
4,5,5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,"[blackberry, raspberry, green, herb, full_bodi...","[[-1.6322763, -2.6095781, 1.1783365, -2.78001,...",13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108872,129965,129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann,"[rich, dry, fresh, ripe, white, fruit, pear, a...","[[-1.0220966, -3.155178, -0.23385222, 0.952300...",9
108873,129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),"[honeysuckle, cataloupe, sweet, juicy, tart, t...","[[-4.227007, -3.1213841, 0.016722424, 2.088082...",9
108874,129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation,"[baked, cherry, cocoa, coconut, soft, fruit, c...","[[-0.38147363, -3.3817122, 2.0994532, -3.18786...",7
108875,129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss,"[dry, crisp, weight, powerful, spice, baked_ap...","[[-1.0713768, -2.2569797, 3.7971995, 1.7632971...",6


In [12]:
rv = wine_reviews_mincount.review_vector

In [20]:
rv[0][0].dtype

dtype('float32')

In [55]:
def descriptors_to_best_match_wines_v2(list_of_descriptors, wineset=wine_reviews_mincount, number_of_suggestions=10):
    weighted_review_terms = []
    for term in list_of_descriptors:
        if term not in dict_of_tfidf_weightings:
            if term not in descriptor_mapping.index:
                print('choose a different descriptor from', term)
                continue
            else:
                term = descriptor_mapping['normalized'][term]
        tfidf_weighting = dict_of_tfidf_weightings[term]
        word_vector = wine_word2vec_model.wv.get_vector(term).reshape(1, 300)
        weighted_word_vector = tfidf_weighting * word_vector
        weighted_review_terms.append(weighted_word_vector)
    review_vector = sum(weighted_review_terms)

    distance, indice = model_knn.kneighbors(review_vector, n_neighbors=len(wine_reviews_mincount))
    distance_list = distance[0].tolist()[1:]
    indice_list = indice[0].tolist()[1:]

    n = 1
    for d, i in zip(distance_list, indice_list):
        if i in wineset.index:
            wine_name = wineset['title'][i]
            wine_descriptors = wineset['normalized_descriptors'][i]
            print('Suggestion', str(n), ':', wine_name, 'with a cosine distance of', "{:.3f}".format(d))
            print('This wine has the following descriptors:', wine_descriptors)
            print('')
            n+=1
        if n == number_of_suggestions + 1:
            break

In [48]:
descriptors_to_best_match_wines_v2(list_of_descriptors=descriptors, number_of_suggestions=5)

Suggestion 1 : Domaine Faiveley 2011 Les Villeranges  (Rully) with a cosine distance of 0.084
This wine has the following descriptors: ['fresh', 'fruit', 'citrus', 'pear', 'apple', 'wood', 'round']

Suggestion 2 : Domaine Ostertag 2010 Barriques Pinot Blanc (Alsace) with a cosine distance of 0.088
This wine has the following descriptors: ['oak', 'wood', 'toast', 'pear', 'honey', 'apple']

Suggestion 3 : A.R. Lenoble  NV Intense Brut  (Champagne) with a cosine distance of 0.110
This wine has the following descriptors: ['wood', 'rich', 'ripe', 'fruit', 'pear', 'crisp', 'apple', 'apricot']

Suggestion 4 : Domaine de la Garenne 2008  Mâcon-Azé with a cosine distance of 0.120
This wine has the following descriptors: ['wood', 'fruit', 'fresh', 'grapefruit', 'pear', 'bright', 'lemon', 'honey', 'wood']

Suggestion 5 : Wheelhouse 2011 Chardonnay (Sonoma County) with a cosine distance of 0.126
This wine has the following descriptors: ['ripe', 'apple', 'pear', 'tropical_fruit', 'fruit', 'oak']



In [71]:
wineset = wine_reviews_mincount.copy()
# condition mask
# mask = wineset['country'] == 'China'
mask = wineset['winery'] == 'Concha y Toro'
 
# new dataframe with selected rows
wineset = pd.DataFrame(wineset[mask])
 
wineset


,level_0,index,country,description,designation,points,price,province,region_1,title,variety,winery,normalized_descriptors,review_vector,descriptor_count
970,1135,1135,Chile,Prickly aromas of nettle are simple as well as...,Reserva Casillero del Diablo,84,11.0,Chile,NaN,Concha y Toro 2015 Reserva Casillero del Diabl...,Sauvignon Blanc,Concha y Toro,"[prickly, nettle, diluted, fresh, green, citru...","[[-3.4539583, -4.7219796, -2.1064138, -2.35390...",8
979,1150,1150,Chile,"Blocky, stewy, raisiny aromas of nonspecific b...",Reserva Casillero del Diablo,83,11.0,Central Valley,NaN,Concha y Toro 2014 Reserva Casillero del Diabl...,Carmenère,Concha y Toro,"[chunky, raisin, fruit, green, jam, prune_rais...","[[-2.2489653, -1.4482371, 3.5228877, -4.837628...",9
1306,1519,1519,Chile,The nose is an attractive mix of cassis and mi...,Terrunyo Vineyard Selection Block Las Terrazas...,92,38.0,Maipo Valley,NaN,Concha y Toro 2009 Terrunyo Vineyard Selection...,Cabernet Sauvignon,Concha y Toro,"[cassis, minerality, racy, dense, soft, juicy,...","[[-1.5547233, -2.490401, 2.7303433, -3.894609,...",10
1755,2078,2078,Chile,We won't call this the best Terrunyo SB to dat...,Terrunyo El Triángulo Vineyard Block: 28,90,25.0,Casablanca Valley,NaN,Concha y Toro 2008 Terrunyo El Triángulo Viney...,Sauvignon Blanc,Concha y Toro,"[clean, minerality, lime, passion_fruit, green...","[[-4.5979714, -2.9245827, -0.1470171, 0.845803...",9
2294,2719,2719,Chile,This Merlot excels beyond its bargain classifi...,Xplorador,87,7.0,Rapel Valley,NaN,Concha y Toro 2002 Xplorador Merlot (Rapel Val...,Merlot,Concha y Toro,"[fruit, flower, plum, chocolate, cream, blackb...","[[-2.7303565, -3.8081338, 3.2226987, -1.484918...",9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106266,126818,126818,Chile,"An opening of dense black-fruit, spice, violet...",Marques de Casa Concha,90,22.0,Puente Alto,NaN,Concha y Toro 2010 Marques de Casa Concha Cabe...,Cabernet Sauvignon,Concha y Toro,"[dense, black_fruit, spice, violet, toast, plu...","[[-2.127777, -2.5323677, 3.782927, -2.5020392,...",15
106781,127499,127499,Chile,"Ripe and ready, with powerful dark fruit. This...",Xplorador,85,8.0,Maipo Valley,NaN,Concha y Toro 2001 Xplorador Cabernet Sauvigno...,Cabernet Sauvignon,Concha y Toro,"[ripe, powerful, dark, fruit, sugar, fruit, ripe]","[[-0.94509727, -1.2340596, 2.7739387, 0.659942...",7
107139,127926,127926,Chile,"The source for this wine is now Limarí, and it...",Marques de Casa Concha,86,19.0,Limarí Valley,NaN,Concha y Toro 2008 Marques de Casa Concha Char...,Chardonnay,Concha y Toro,"[oak, toast, stemmy, apple, peach, racy, sour,...","[[-0.9904996, -1.8736748, -2.3601463, 0.893406...",10
107439,128285,128285,Chile,Blackened spice aromas are affixed to blackber...,Gravas del Maipo,92,150.0,Buin,NaN,Concha y Toro 2011 Gravas del Maipo Syrah (Buin),Syrah,Concha y Toro,"[spice, blackberry, herb, full_bodied, plum, b...","[[-0.97891295, -3.9138598, 1.3336896, -3.53484...",13


In [72]:
descriptors_to_best_match_wines_v2(list_of_descriptors=descriptors, wineset=wineset, number_of_suggestions=5)

Suggestion 1 : Concha y Toro 2012 Marques de Casa Concha Chardonnay (Limarí Valley) with a cosine distance of 0.220
This wine has the following descriptors: ['resin', 'tropical_fruit', 'fruit', 'apple', 'melon', 'oak', 'plump', 'oak', 'spice', 'apple', 'melon', 'wood']

Suggestion 2 : Concha y Toro 2001 Amelia Limited Release Chardonnay (Casablanca Valley) with a cosine distance of 0.220
This wine has the following descriptors: ['oak', 'plump', 'stemmy', 'oak', 'flower', 'popcorn', 'tree_fruit', 'ripe', 'apple', 'banana', 'soft', 'cream', 'lemon', 'wood']

Suggestion 3 : Concha y Toro 2013 Marques de Casa Concha Chardonnay (Limarí Valley) with a cosine distance of 0.259
This wine has the following descriptors: ['resin', 'flower', 'round', 'wood', 'vanilla', 'tropical_fruit', 'fruit', 'chalk', 'minerality', 'tree_fruit', 'melon', 'spice', 'oak', 'savory', 'spice']

Suggestion 4 : Concha y Toro 2001 Xplorador Chardonnay (Casablanca Valley) with a cosine distance of 0.272
This wine has th

In [43]:
wine_reviews_mincount.country.unique()

array(['Italy', 'Portugal', 'US', 'Spain', 'Germany', 'France',
       'Argentina', 'Chile', 'Australia', 'Austria', 'South Africa',
       'New Zealand', 'Israel', 'Hungary', 'Greece', 'Romania', 'Mexico',
       'Canada', nan, 'Turkey', 'Czech Republic', 'Slovenia',
       'Luxembourg', 'Croatia', 'Georgia', 'Uruguay', 'England',
       'Lebanon', 'Serbia', 'Brazil', 'Moldova', 'Morocco', 'Peru',
       'India', 'Bulgaria', 'Cyprus', 'Armenia', 'Switzerland',
       'Bosnia and Herzegovina', 'Ukraine', 'Slovakia', 'Macedonia',
       'China', 'Egypt'], dtype=object)

In [70]:
wine_reviews_mincount.winery.value_counts()


Williams Selyem       202
Testarossa            199
Wines & Winemakers    168
Concha y Toro         159
Louis Latour          150
                     ... 
Casir Dos Santos        1
Huntaway Reserve        1
Onward                  1
Vriesenhof              1
Penedo Borges           1
Name: winery, Length: 15603, dtype: int64